## Trip Advisor System


This project will work on a practical solution for the classification of negative and positive tweets for airlines and will work on how we can and how we will use this model in the future we will carry out some analyzes on tweets and establish a small recommendation to users on the best companies and customer recommendation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy
import seaborn as sns;
sns.set(style="ticks", color_codes=True)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

## Exploraty data analysis

importing and get First 5 Rows of the data

In [ ]:
Tweet= pd.read_csv("/kaggle/input/twitter-airline-sentiment/Tweets.csv")
Tweet.head()

in This step we try to measure what's columns have the higest column has missed values 

In [ ]:
(len(Tweet)-Tweet.count())/len(Tweet)


Hieghst Columns Missing Values Percentage  :
- negativereason_gold             99.8%
- airline_sentiment_gold          99.8%
- tweet_coord                     93%

and in next step we will remove it we didn't need it anymore
             



In [ ]:
del Tweet['tweet_coord']
del Tweet['airline_sentiment_gold']
del Tweet['negativereason_gold']

At this point, it might be helpful for us visualize a distribution of the article sentiment confidence and negative  confidence to see how skewed our average might be by outliers. Let's generate another plot to take a look:

In [ ]:


ax=Tweet['airline_sentiment_confidence'].plot(kind='hist', bins=50, fontsize=14, ,figsize=(12,7))
ax.set_title('Article Length in Words\n', fontsize=20)
ax.set_ylabel('Frequency', fontsize=18)
ax.set_xlabel('Number of Words', fontsize=18);

In [ ]:


ax=Tweet['negativereason_confidence'].plot(kind='hist', bins=50, fontsize=14, figsize=(12,7))
ax.set_title('Article Length in Words\n', fontsize=20)
ax.set_ylabel('Frequency', fontsize=18)
ax.set_xlabel('Number of Words', fontsize=18);

so i see we can put a threshold right there :
- %80 for airline sentiment confidence 
- and if you want to make it classification problem with negative reasons just put another threshold at 65%<br><br>
if confidence is lower than threshold drop all rows why ? because its called outlier and mabye make your model explode on your face

In [ ]:
Best_confidence = Tweet[Tweet['airline_sentiment_confidence'] > .80]
print('you have dropped :'+ str((round((1 - Best_confidence.shape[0]/Tweet.shape[0]) * 100 )))+ '% of your data which has less that 80% confidence')
Tweet = Best_confidence

 we’ll add the result as a new column that contains the number of unique words in each article.

In [ ]:
Tweet['uniq_wds'] = Tweet['text'].str.split().apply(lambda x: len(set(x)))
Tweet['uniq_wds'].mean()

In [ ]:
ax=Tweet['uniq_wds'].plot(kind='hist', bins=50, fontsize=14, figsize=(12,7))
ax.set_title('Unique Words Per Article\n', fontsize=20)
ax.set_ylabel('Frequency', fontsize=18)
ax.set_xlabel('Number of Unique Words', fontsize=18);

Here we will create our First Helper Function plotBars wich help us to visualize unique class counts foreach column :

In [ ]:
def plotBars(col):
    sns.countplot(x=col, hue=col, data=Tweet ,palette="ch:.25")


Let's Visualize Airline Sentiment :

In [ ]:
plt.figure(figsize=(12,7))
plotBars('airline_sentiment')

8000 negative tweets and more than 3000 neutral tweets and also more than 2000 Positive Tweets <br> 
Lets Visualize Air Lines  Data :

In [ ]:
plt.figure(figsize=(12,7))
plotBars('airline')

Here we will Visualize Each Airlane sentiment distubation in it's 3 classes


In [ ]:
#sns.catplot( row="airline" ,col="airline_sentiment" ,kind="count", palette="ch:.25", data=Tweet);
plt.figure(figsize=(12,7))
sns.countplot(x="airline", hue="airline_sentiment",palette="ch:.25", data=Tweet)


we can observe what is more and lowest data

In [ ]:
NR_Count=dict(Tweet['negativereason'].value_counts(sort=False))


In [ ]:
NR_Count

In [ ]:
def NR_Count(Airline):
    if Airline=='All':
        df=Tweet
    else:
        df=Tweet[Tweet['airline']==Airline]
    count=dict(df['negativereason'].value_counts())
    Unique_reason=list(Tweet['negativereason'].unique())
    Unique_reason=[x for x in Unique_reason if str(x) != 'nan']
    Reason_frame=pd.DataFrame({'Reasons':Unique_reason})
    Reason_frame['count']=Reason_frame['Reasons'].apply(lambda x: count[x])
    return Reason_frame

In [ ]:
NR_Count('All')

Here we Visualize the biggest reasons that may lead to negative views and we will conclude that the biggest problems are customer service and problems in the flight process and delays and bad leadership.
Well we can summarize the problem in that the password for success is customer service because it is possible where there is a tweet reported that there are problems because of what but also added that the customer service was bad

In [ ]:
g = sns.catplot(x="Reasons", y="count", palette="ch:.25" ,kind="bar",data=NR_Count('All'))
g.set_xticklabels(rotation=90)


That way we will present some words that are repeated in most tweets and we will notice that all things are related to either time management or personal belongings or ask for help from the community
In fact, I believe that the word help is being used to ask to expose the company and the greater the number of retweets, the more it becomes tragic

Well we have a simple conclusion here that the method of complaint is by exposing the company and present the position of the evaluation to the public and this means that we need to pay attention to the tweets with large numbers of re-tweet because it may contain some negative words, especially if they are largely negative

In [ ]:
from wordcloud import WordCloud,STOPWORDS
df=Tweet[Tweet['airline_sentiment']=='negative']
words = ' '.join(df['text'])
cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])
wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color='black',
                      width=3000,
                      height=2500
                     ).generate(cleaned_word)
plt.figure(1,figsize=(12, 12))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

Words of thanks and gratitude and some keywords that are repeated in very large quantities may express positive in dealing with the company and recommendations to deal with in the future

In [ ]:
from wordcloud import WordCloud,STOPWORDS
df=Tweet[Tweet['airline_sentiment']=='positive']
words = ' '.join(df['text'])
cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])
wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color='black',
                      width=3000,
                      height=2500
                     ).generate(cleaned_word)
plt.figure(1,figsize=(12, 12))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# NLP - Data Proccessing 

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

1- we trying to made Function to Convert Tweets to Words if it's not in stopword <br>
- what is stopword ?
in natural language processing, useless words (data), are referred to as stop words. Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine has been programmed to ignore, both when indexing entries for searching and when retrieving them as the result of a search query

In [ ]:
def tweet_to_words(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words )) 

2 - Here we trying to make it cleanTweet and That's mean we need it to be just in english language 

In [ ]:
def clean_tweet_length(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return(len(meaningful_words)) 

3 - in this step we start proccessing data between negative and positive but in numeric way becuase our machine not know how to dail with strings just with numbers that representation of unique classess

In [ ]:
Tweet['sentiment']=Tweet['airline_sentiment'].apply(lambda x: 0 if x=='negative' else 1)
Tweet['clean_tweet']=Tweet['text'].apply(lambda x: tweet_to_words(x))
Tweet['Tweet_length']=Tweet['text'].apply(lambda x: clean_tweet_length(x))

4- Let's apply Train and Test Split with 80% for training and 20% for testing and select tweets randomly

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(Tweet,test_size=0.2,random_state=42)

![](http://)5 - Apply Clean Tweet Function

In [ ]:
train_clean_tweet=[]
for tweet in train['clean_tweet']:
    train_clean_tweet.append(tweet)
test_clean_tweet=[]
for tweet in test['clean_tweet']:
    test_clean_tweet.append(tweet)


6 - We have a clean data here but we need to make it to be Vectorizerd by eachword 
 
 - Vectorized by eachword : Convert a collection of text documents to a matrix of token counts
 - Override the string tokenization step while preserving the preprocessing and n-grams generation steps.



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(analyzer = "word")
train_features= v.fit_transform(train_clean_tweet)
test_features=v.transform(test_clean_tweet)

 ## Machine Learning Lab:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

In [ ]:
Classifiers = [
    LogisticRegression(C=0.000000001,solver='liblinear',max_iter=200),
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=200),
    AdaBoostClassifier(),
    GaussianNB()]

In [ ]:
dense_features=train_features.toarray()
dense_test= test_features.toarray()
Accuracy=[]
Model=[]
for classifier in Classifiers:
    try:
        fit = classifier.fit(train_features,train['sentiment'])
        pred = fit.predict(test_features)
    except Exception:
        fit = classifier.fit(dense_features,train['sentiment'])
        pred = fit.predict(dense_test)
    accuracy = accuracy_score(pred,test['sentiment'])
    Accuracy.append(accuracy)
    Model.append(classifier.__class__.__name__)
    print('Accuracy of '+classifier.__class__.__name__+'is '+str(accuracy))    

Well, the random forest is the best algorithm so far and performs almost 80%. Well that's an acceptable figure as a first step But we'll improve it Using Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
classifier = RandomForestClassifier(n_estimators=200)
all_accuracies = cross_val_score(estimator=classifier, X=train_features, y=train['sentiment'], cv=5)


How Much Accuracy and Variance in our Model :


In [ ]:
print('Acc',all_accuracies.mean())
print('Std',all_accuracies.std())


Note that the prevalence when multiplied by 100% is very low, but 80% is not an appropriate ratio at all, so you can try to use some other strategies like GridSearch with Cross Validation Using Random Forest Classifer !



# Deep Learning Lab :

Well, the way we deal with data in the case of deep learning will be a little different because we will use Recurrent neural networks based on iterative patterns, sequence data , whether in all sentences or in subjects or in the whole tweets, so we have to simplify it in another way

-

1 - Importing Tools :

In [ ]:
import numpy as np
import pandas as pd
import re
#from sentiment_utils import *
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
#np.random.seed(1)
from sklearn.model_selection import train_test_split

Helper Function :
- Remove Function
- Remove Tag Function

In [ ]:
def remove_stopwords(input_text):
    '''
    Function to remove English stopwords from a Pandas Series.
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    stopwords_list = stopwords.words('english')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words)

def remove_mentions(input_text):
    '''
    Function to remove mentions, preceded by @, in a Pandas Series
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    return re.sub(r'@\w+', '', input_text)

In [ ]:
Tweet.head()

In [ ]:
#cleaning Data
train_df = Tweet[['text', 'airline_sentiment']]
train_df.text = Tweet.text.apply(remove_mentions)
train_df.loc[:,'sentiment'] = train_df.airline_sentiment.map({'negative':0,'neutral':1,'positive':2})
train_df = train_df.drop(['airline_sentiment'], axis=1)


1- Define max length of all tweet

In [ ]:
#This step is to find the maximun length of the input string so as to fed the neural net with same length
raw_docs_train = train_df["text"].values
sentiment_train = train_df['sentiment'].values

maxLen = len(max(raw_docs_train, key=len).split())
maxLen

2- Splitting data 

In [ ]:
#For our model we need to split our training dataset into test dataset. This is actually dev set for getting the loss
X_train, X_test, Y_train, Y_test = train_test_split(raw_docs_train, sentiment_train, 
                                                  stratify=sentiment_train, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)
print('Train data samples:', X_train.shape)
print('Test data samples:', X_test.shape)
assert X_train.shape[0] == Y_train.shape[0]
assert X_test.shape[0] == Y_test.shape[0]

3 - Convert the labels to One hot encoding vector for softmax for neural network

In [ ]:

#Y_oh_train = convert_to_one_hot(Y_train, C = num_labels)
num_labels = len(np.unique(sentiment_train))
Y_oh_train = np_utils.to_categorical(Y_train, num_labels)
Y_oh_test = np_utils.to_categorical(Y_test, num_labels)
print(Y_oh_train.shape)


[GloVe](https://nlp.stanford.edu/projects/glove/) is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.


4 - Now we need Glove Vectors for Word which is available online 

In [ ]:

# load the GloVe vectors in a dictionary:

def read_glove_vecs(glove_file):
    with open(glove_file, encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('../input/glove-global-vectors-for-word-representation/glove.6B.50d.txt')

-- GloVe is you here ?


In [ ]:
word = "bad"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

Glove : Loaii why are you get me out from stanford ?<br>
Loaii : we have some work with some tweets ...<br>
Glove : Tweets agaaaain !!<br>


5- This function convert array of strings into array of Indices of word in the voacab

In [ ]:

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =[word.lower().replace('\t', '') for word in X[i].split(' ') if word.replace('\t', '') != '']
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            try:
                X_indices[i, j] = word_to_index[w]
            except: 0
            # Increment j to j + 1
            j = j+1
    
    return X_indices

6 - Create Keras Embedding layer

In [ ]:
# Create Keras Embedding layer
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

7- Define LSTM Model 

In [ ]:

def ltsm_model(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the ltsm_model model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices =  Input(shape=input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 512-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(512, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.3
    X = Dropout(0.3)(X)
     # Propagate X trough another LSTM layer with 128-dimensional hidden state
    X = LSTM(256, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)
    
    # Propagate X trough another LSTM layer with 512-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(512, return_sequences=False)(X)
    # Add dropout with a probability of 0.3
    X = Dropout(0.3)(X)
    
    
    # Propagate X through a Dense layer with softmax activation to get back a batch of 3-dimensional vectors.
    X = Dense(3, activation=None)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=[sentence_indices], outputs=X)
    
    ### END CODE HERE ###
    
    return model

8 - Summary our code

In [ ]:
model = ltsm_model((maxLen,), word_to_vec_map, word_to_index)
model.summary()

9 - Compiling our ooptimizers :

In [ ]:
from keras.optimizers import Adam,Adagrad,rmsprop
adam = Adam(lr=0.01, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


10 - convert sentences to indices

In [ ]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
print(X_train_indices.shape)

11 - at first we define earlystopping that observe our validation data lose and get restore best weights after that if there is any improve of val_loss on maximum 30 epoch it will start and our mode is auto
12- we fit our model with train data after that we run it on 512 batch size with 120 epoch 

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', restore_best_weights = True ,min_delta=0, patience=30, verbose=0, mode='auto')

model.fit(X_train_indices, y=Y_oh_train, batch_size=512, epochs=120, 
          verbose=1, validation_data=(X_test_indices, Y_oh_test), callbacks=[earlystop])

13 - That's Test Model make testing appoarch on any data model 

In [ ]:
def TestModel(stringList):
        classess = ['negative','neutral','positive']
        results=[]
        if len(stringList) == 1:
            x_test = np.array(stringList)
            X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
            index = np.argmax(model.predict(X_test_indices))
            return x_test[0] +' is a '+  classess[index]+' word'
        for i,x in enumerate(stringList):
            x = [x]
            x_test = np.array(x)
            X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
            index = np.argmax(model.predict(X_test_indices))
            results.append(x_test[0] +' is a '+  classess[index]+' word')
        return results
            
        

# Testing Our Model :

In [ ]:
TestModel(['Nice AirPlance Service I love Them ', # Positive
           'bad AirPlance', # negative
           'What The Fuckk Plane' , # negative
           'my Flight is so late and its about 1 hour why ?', # negative
           'okay its fine flight , thank you ',# Postive
           'I lost my Bag and and They Back it to me So Thanks ', # Negative
           'okay' # neutral
          ])

7/7 That's Awesome and here we are very thanks and Let's Try a tweet contains a small story:

# Bouns Idea :

In [ ]:
story = 'The flight was delayed at first but I felt so bad because I waited for more than three hours, the flight crew that he give us with\
free meals during the flight that was awesome , I know we paid for the meal before but in fact they were more than \
five meals  and i feel so But I am quite satisfied with the trip , My best trip ever but I hate them ..\
Im kidding I loved that service very much I hope you try it' 
# Negative Story + Kidding + Positive and Recommendation Can our model find it ?

In [ ]:
TestModel([story])

Unfortunately, as you can see, he cannot distinguish us as human beings as to what was a good event and became a bad event or a bad event and became a good thing.
At each interval we will consider it as a separate sentence and we will predict the person's condition
For example, if the percentage is less than 40% of the total sentences, we will contact the author of the tweet to solve the problem immediately
in next example we had more 50% 2 positive and 2 negative of 1 stroy so , we made our filter before at 40% of positive label so thats positive and at the next story splited with comma we have 50% pos acc so its positive 

In [ ]:
TestModel(story.split(','))